In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from random import shuffle
from tqdm import tqdm


/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Reading labels

In [2]:
import pandas as pd
y=pd.read_csv('labels.csv')

### Loading Images and assigning them classes and resizing them

In [3]:
img_width = 120
img_height = 120
images=[]
classes=[]
#load training images
for i in range(120):
    for x in os.listdir("dog_train"):
        if i+1<=len(os.listdir('dog_train/'+x)) :
            img = cv2.imread('dog_train/'+x+'/'+os.listdir(os.path.join("dog_train",x))[i])
            resize_image = cv2.resize(img, (img_width,img_height))
            images.append(resize_image)
            classes.append(x)

### Creating one hot for classes

In [4]:
y = np.array(pd.get_dummies(classes,dtype="float32"))

## Image Augmentation
### This function will take a list of images and for each image there is 50% chance it will get passed to placeholder without any modification or it will get augmented in any of the 8 augmentations:  Flip (updown or left/right), Dilation, Blur, Rotation, Shear,Translation, RGB.

In [5]:
def augment(image):
    Aug_operation= ['flip_horz','flip_up','blur','dilation','rgb_','rotate','translate','sheer']
    img = []
    out=[]
    aug_img = []
    img_width = (image[0]).shape[0]
    img_height = (image[0]).shape[1]
    n_op = len(Aug_operation)
    for j in range(len(image)): 
        apply_aug =np.random.randint(0,2)
        if apply_aug == 0:
            aug_img = image[j]
        else:
            rand_int= np.random.randint(0,n_op)
            rows,cols,ch = image[0].shape 
            if rand_int%n_op == 0:
                aug_img = (cv2.flip(image[j],1))
            elif rand_int%n_op ==1:        
                aug_img=(cv2.flip(image[j],0))
            elif rand_int%n_op ==2:
                aug_img=(cv2.blur(image[j],(3,3)))
            elif rand_int%n_op ==3:
                kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
                aug_img=(cv2.dilate(image[j],kernel_ellipse,iterations=1))
            elif rand_int%n_op ==4:
                aug_img=cv2.cvtColor(image[j],cv2.COLOR_BGR2RGB)
            elif rand_int%n_op ==5:
                angle= np.random.uniform(-1,1)*90
                Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
                aug_img=(cv2.warpAffine(image[j],Rot_M,(cols,rows)))
            elif rand_int%n_op ==6:
                x_shift = np.random.uniform(-1,1)*img_width/8
                y_shift = np.random.uniform(-1,1)*img_height/8
                Trans_M = np.float32([[1,0,x_shift],[0,1,y_shift]])
                aug_img=(cv2.warpAffine(image[j],Trans_M,(cols,rows)))         
            elif rand_int%n_op ==7:           
                pts1 = np.float32([[5,5],[20,5],[5,20]])
                pts2 = np.float32([[10,10],[25,17],[8,20]])
                M = cv2.getAffineTransform(pts1,pts2)
                aug_img = cv2.warpAffine(image[j],M,(cols,rows))
        out.append(aug_img)
    return out

### Weight Initialization

In [6]:
def init_weight(shape):
    w=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return w

### Bias Initialization

In [7]:
def init_biases(shape):
    b=tf.Variable(tf.constant(0.1,shape=shape))
    return b

### Convolution

In [8]:
def conv2d(X1,w):
    return tf.nn.conv2d(X1,w,strides=[1,1,1,1],padding='SAME')

### Max Pooling

In [9]:
def max_pool(X1):
    return tf.nn.max_pool(X1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### Applies Convolution -> Add Bias -> ReLU Activation 

In [10]:
def convolution_layer(X1,shape):
    w=init_weight(shape)
    b=init_biases([shape[3]])
    return tf.nn.relu(conv2d(X1,w)+b)

### Function for Fully Connected Layer

In [11]:
def fully_connected_layer(X1,size):
    feature=int(X1.get_shape()[1])
    w=init_weight([feature,size])
    b=init_biases([size])
    return tf.matmul(X1,w)+b

### Placeholder for Input and label (Image Size=[120,120])

In [12]:
X1=tf.placeholder(tf.float32,shape=[None,120,120,3])
y_true=tf.placeholder(tf.float32,shape=[None,120])

### Convolution(3,3,channel(3->16)) -> Max Pooling(2,2) -> Convolution(3,3,channel(16->32)) -> Max Pooling(2,2)

In [13]:
conv_1=convolution_layer(X1,shape=[3,3,3,16])
max_pool1=max_pool(conv_1)
conv_2=convolution_layer(max_pool1,shape=[3,3,16,32])
max_pool2=max_pool(conv_2)

In [14]:
# lrn1=tf.nn.local_response_normalization(
#     max_pool2,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

### Convolution(3,3,channel(32->64)) -> Max Pooling(2,2)

In [15]:
conv_3=convolution_layer(max_pool2,shape=[3,3,32,64])
max_pool3=max_pool(conv_3)
max_pool3

<tf.Tensor 'MaxPool_2:0' shape=(?, 15, 15, 64) dtype=float32>

### Convolution(3,3,channel(64->128))

In [16]:
conv_4=convolution_layer(max_pool3,shape=[3,3,64,128])
conv_4

<tf.Tensor 'Relu_3:0' shape=(?, 15, 15, 128) dtype=float32>

In [17]:
#max_pool4=max_pool(conv_4)

In [18]:
# lrn2=tf.nn.local_response_normalization(
#     max_pool4,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

In [19]:
#conv_5=convolution_layer(lrn2,shape=[3,3,64,128])
#max_pool5=max_pool(conv_5)
#conv_6=convolution_layer(max_pool5,shape=[3,3,128,256])
#max_pool6=max_pool(conv_6)
#hold_prob = tf.placeholder(tf.float32)
#full_one_dropout = tf.nn.dropout(max_pool6,keep_prob=hold_prob)

### Flatten

In [20]:
depth_flat=tf.reshape(conv_4,[-1,128*225])

### Fully Connected Layer -> Batch_Normalization -> Prediction

In [21]:
fcn1=(fully_connected_layer(depth_flat,2048))
bn1=tf.nn.relu(tf.layers.batch_normalization(fcn1))
#fcn2=tf.nn.relu(fully_connected_layer(bn2,1024))
# hold_prob = tf.placeholder(tf.float32)
# full_2_dropout = tf.nn.dropout(bn1,keep_prob=hold_prob)
#fcn2=tf.nn.relu(fully_connected_layer(full_2_dropout,512))
#full_3_dropout = tf.nn.dropout(fcn2,keep_prob=hold_prob)
y_pred=(fully_connected_layer(bn1,120))
y_pred

<tf.Tensor 'add_5:0' shape=(?, 120) dtype=float32>

### Cross Entropy Loss and Training with Adam Optimizer

In [22]:
with tf.name_scope('Loss'):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
with tf.name_scope('Adam'):
    train_op=tf.train.AdamOptimizer().minimize(loss_op)    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy

In [23]:
with tf.name_scope('Accuracy'):
    prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Separating Training (9600) and Test Set (614)

In [24]:
images_train=images[:9600]
y_train=y[:9600]
images_test=images[9601:10214]
y_test=y[9601:10214]

### Graph and Summary

In [25]:
tf.summary.scalar("loss", loss_op)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()
saver=tf.train.Saver()

### Running the Session (21 Epochs and Batch Size=32).Training accuracy varying from 30% to 70% . Maximium Test Accuracy 3.26%

In [27]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./output1', sess.graph)
    steps=300
    sess.run(tf.global_variables_initializer())
    for j in range(60):        
        for i in range(steps):         
            batch_x=augment(images_train[32*i:32*(i+1)])
            batch_y=(y_train[32*i:32*(i+1)])            
            print(sess.run([train_op,loss_op,accuracy],feed_dict={X1:batch_x,y_true:batch_y}))
            _, c, summary = sess.run([train_op, loss_op, merged_summary_op],feed_dict={X1: batch_x, y_true: batch_y})
            summary_writer.add_summary(summary,i)
        print('epoch',j,' loss,accuracy:',sess.run([loss_op,accuracy],feed_dict={X1:images_test,y_true:y_test}))    

[None, 5035.8667, 0.0]
[None, 7807.47, 0.0]
[None, 5134.566, 0.0]
[None, 2239.7495, 0.0]
[None, 1084.0426, 0.0]
[None, 407.72687, 0.0]
[None, 298.76807, 0.0]
[None, 245.4344, 0.0]
[None, 159.04141, 0.03125]
[None, 94.69111, 0.0]
[None, 85.922066, 0.0625]
[None, 78.66725, 0.0]
[None, 62.90693, 0.0]
[None, 47.693047, 0.0]
[None, 32.91554, 0.0]
[None, 40.589066, 0.0]
[None, 25.69508, 0.03125]
[None, 26.907015, 0.03125]
[None, 24.5886, 0.03125]
[None, 26.337555, 0.0]
[None, 20.857468, 0.0]
[None, 22.618734, 0.0]
[None, 15.63473, 0.0]
[None, 16.812424, 0.03125]
[None, 16.125814, 0.03125]
[None, 16.77217, 0.0]
[None, 12.72946, 0.03125]
[None, 9.416227, 0.03125]
[None, 13.588864, 0.0]
[None, 10.124533, 0.0]
[None, 7.9912267, 0.0]
[None, 9.268136, 0.0]
[None, 11.766151, 0.0]
[None, 9.539251, 0.03125]
[None, 8.2798815, 0.0]
[None, 7.289573, 0.0]
[None, 8.64192, 0.03125]
[None, 7.721718, 0.0]
[None, 9.475471, 0.0]
[None, 6.0508018, 0.0]
[None, 10.12406, 0.0]
[None, 7.2537594, 0.0]
[None, 7.38736

[None, 4.88865, 0.0]
[None, 4.83434, 0.0]
[None, 4.8101163, 0.0]
[None, 4.753039, 0.0]
[None, 4.8128734, 0.0]
[None, 4.8099995, 0.0]
[None, 4.8842797, 0.0]
[None, 4.7159057, 0.0]
[None, 4.9384117, 0.03125]
[None, 4.848216, 0.0]
[None, 4.848939, 0.0]
[None, 4.804493, 0.0]
[None, 4.759033, 0.03125]
[None, 4.808912, 0.0]
[None, 4.707671, 0.03125]
[None, 4.8976727, 0.0]
[None, 4.7777195, 0.0]
[None, 4.6514816, 0.03125]
[None, 4.7453947, 0.03125]
[None, 4.8013406, 0.03125]
[None, 4.7156906, 0.03125]
[None, 4.818612, 0.03125]
[None, 4.825027, 0.0]
[None, 4.7758856, 0.03125]
[None, 4.769656, 0.03125]
[None, 4.80208, 0.0]
[None, 4.7972155, 0.0]
[None, 4.7407255, 0.03125]
[None, 4.806488, 0.03125]
[None, 4.9157143, 0.0]
[None, 4.8913193, 0.0]
[None, 4.786972, 0.0]
[None, 4.868991, 0.0]
[None, 4.7327523, 0.03125]
[None, 4.8614726, 0.03125]
[None, 4.761685, 0.03125]
[None, 4.8431935, 0.0]
[None, 4.7711163, 0.0625]
[None, 4.907072, 0.03125]
[None, 4.7149906, 0.03125]
[None, 4.651138, 0.03125]
[Non

[None, 4.694007, 0.0625]
[None, 4.9606667, 0.0]
[None, 4.652117, 0.0625]
[None, 4.7986116, 0.0]
[None, 5.055944, 0.03125]
[None, 4.7822347, 0.0]
[None, 4.757574, 0.03125]
[None, 4.685397, 0.03125]
[None, 4.8600297, 0.0]
[None, 4.527, 0.0625]
[None, 4.9101467, 0.0]
[None, 4.6715937, 0.0625]
[None, 4.8481092, 0.0]
[None, 5.013912, 0.03125]
[None, 4.797578, 0.03125]
[None, 4.789501, 0.0]
[None, 4.6185064, 0.03125]
[None, 4.520859, 0.09375]
[None, 4.688796, 0.03125]
[None, 4.682296, 0.0625]
[None, 4.807577, 0.0]
[None, 4.8012886, 0.03125]
[None, 4.6260276, 0.0625]
[None, 4.8497114, 0.03125]
[None, 4.858818, 0.03125]
[None, 4.7050495, 0.03125]
[None, 4.807875, 0.03125]
[None, 4.88473, 0.0]
[None, 4.554027, 0.09375]
[None, 4.673278, 0.03125]
[None, 4.6854134, 0.03125]
[None, 4.6407967, 0.03125]
[None, 4.753994, 0.0625]
[None, 4.575304, 0.03125]
[None, 4.5916634, 0.03125]
[None, 4.6022816, 0.03125]
[None, 4.490924, 0.09375]
[None, 4.776263, 0.0]
[None, 4.5659094, 0.0625]
[None, 4.5645537, 0.0

[None, 4.8832417, 0.0]
[None, 4.554983, 0.03125]
[None, 4.7777567, 0.0625]
[None, 4.4085937, 0.125]
[None, 4.787967, 0.03125]
[None, 4.736783, 0.03125]
[None, 5.4513397, 0.03125]
[None, 4.197364, 0.15625]
[None, 4.4908295, 0.0625]
[None, 4.5609946, 0.09375]
[None, 4.7794523, 0.03125]
[None, 4.768967, 0.09375]
[None, 4.590261, 0.03125]
[None, 4.775276, 0.0625]
[None, 4.684883, 0.0625]
[None, 4.67529, 0.03125]
[None, 4.6711197, 0.0]
[None, 4.7963924, 0.0]
[None, 4.702584, 0.03125]
[None, 4.712328, 0.03125]
[None, 4.7929416, 0.0]
[None, 4.8335233, 0.0]
[None, 4.898796, 0.0]
[None, 4.508336, 0.09375]
[None, 4.63407, 0.0625]
[None, 4.5745745, 0.0625]
[None, 5.041778, 0.03125]
[None, 4.832207, 0.03125]
[None, 4.55548, 0.0625]
[None, 4.791267, 0.0]
[None, 4.702734, 0.03125]
[None, 4.630191, 0.03125]
[None, 4.584053, 0.0625]
[None, 4.679937, 0.03125]
[None, 4.646071, 0.03125]
[None, 4.2847424, 0.09375]
[None, 4.838814, 0.0]
[None, 4.5615234, 0.03125]
[None, 4.751913, 0.0]
[None, 4.7713966, 0.0

[None, 4.508849, 0.125]
[None, 4.7921677, 0.0625]
[None, 4.591183, 0.0625]
[None, 4.5191765, 0.09375]
[None, 4.7533674, 0.0625]
[None, 4.6174793, 0.0625]
[None, 4.83964, 0.0]
[None, 4.7679496, 0.0]
[None, 4.594795, 0.03125]
[None, 4.599972, 0.09375]
[None, 4.554285, 0.0625]
[None, 4.613526, 0.0625]
[None, 4.688361, 0.0625]
[None, 4.8176813, 0.0]
[None, 4.506916, 0.0625]
[None, 4.811678, 0.0]
[None, 4.5916204, 0.03125]
[None, 4.4720426, 0.15625]
[None, 4.755191, 0.03125]
[None, 4.571286, 0.0625]
[None, 4.866463, 0.0625]
[None, 4.739834, 0.03125]
[None, 4.7042336, 0.03125]
[None, 4.9016724, 0.0]
[None, 4.7264323, 0.03125]
[None, 4.948022, 0.03125]
[None, 4.647965, 0.03125]
[None, 4.4415274, 0.125]
[None, 4.632468, 0.0625]
[None, 4.5123196, 0.0625]
[None, 4.8227024, 0.0]
[None, 4.711282, 0.0]
[None, 4.8499784, 0.03125]
[None, 4.765005, 0.03125]
[None, 4.5033484, 0.09375]
[None, 4.581444, 0.03125]
[None, 4.9526224, 0.03125]
[None, 4.7240863, 0.03125]
[None, 4.2848988, 0.15625]
[None, 4.617

...

[None, 2.8387222, 0.53125]
[None, 3.3990693, 0.34375]
[None, 2.6747203, 0.40625]
[None, 2.663054, 0.53125]
[None, 3.0091836, 0.375]
[None, 3.1400046, 0.46875]
[None, 2.292348, 0.65625]
[None, 3.3276982, 0.40625]
[None, 3.3362107, 0.375]
[None, 3.3017597, 0.40625]
[None, 2.1281474, 0.5625]
[None, 2.5611787, 0.46875]
[None, 2.6053069, 0.53125]
[None, 2.7008116, 0.5625]
[None, 2.5501778, 0.5]
[None, 3.1248715, 0.40625]
[None, 2.4639688, 0.46875]
[None, 3.041946, 0.5]
[None, 4.381, 0.40625]
[None, 4.20951, 0.21875]
[None, 3.480751, 0.4375]
[None, 3.1887522, 0.375]
[None, 3.416786, 0.375]
[None, 3.2652106, 0.3125]
[None, 3.8470125, 0.34375]
[None, 3.8094068, 0.28125]
[None, 4.170342, 0.25]
[None, 4.801322, 0.125]
[None, 3.4700966, 0.34375]
[None, 3.9112387, 0.28125]
[None, 4.5901227, 0.09375]
[None, 2.869114, 0.375]
[None, 3.4806585, 0.3125]
[None, 4.2959814, 0.15625]
[None, 3.8487854, 0.25]
[None, 2.9523964, 0.40625]
[None, 3.2999353, 0.34375]
[None, 3.9223552, 0.25]
[None, 4.114913, 0.187

[None, 3.007893, 0.4375]
[None, 2.8617399, 0.46875]
[None, 2.7424626, 0.53125]
[None, 3.1715176, 0.40625]
[None, 3.2113147, 0.3125]
[None, 3.2333295, 0.375]
[None, 2.9126444, 0.34375]
[None, 2.6353526, 0.53125]
[None, 4.1217575, 0.3125]
[None, 2.1129606, 0.53125]
[None, 2.8606858, 0.40625]
[None, 3.060131, 0.40625]
[None, 2.4800172, 0.5625]
[None, 3.7300718, 0.34375]
[None, 2.3353825, 0.53125]
[None, 2.7260733, 0.625]
[None, 3.5014803, 0.4375]
[None, 3.9397242, 0.34375]
[None, 3.825183, 0.34375]
[None, 2.768381, 0.46875]
[None, 3.6189237, 0.34375]
[None, 2.40561, 0.5]
[None, 3.5200334, 0.34375]
[None, 3.2971659, 0.3125]
[None, 3.5131564, 0.34375]
[None, 3.338838, 0.40625]
[None, 2.8835568, 0.40625]
[None, 4.400623, 0.34375]
[None, 3.1806433, 0.34375]
[None, 3.4913092, 0.28125]
[None, 2.898592, 0.46875]
[None, 2.5749407, 0.46875]
[None, 3.368864, 0.34375]
[None, 4.243971, 0.1875]
[None, 2.8733406, 0.375]
[None, 3.4285185, 0.375]
[None, 3.3825676, 0.3125]
[None, 3.2071946, 0.46875]
[None

[None, 2.736342, 0.5625]
[None, 2.2153733, 0.59375]
[None, 2.1046023, 0.625]
[None, 3.8345282, 0.4375]
[None, 2.7936015, 0.53125]
[None, 3.1385841, 0.59375]
[None, 2.8178577, 0.4375]
[None, 2.8674867, 0.40625]
[None, 3.3924727, 0.5]
[None, 2.3444138, 0.5625]
[None, 3.2672906, 0.46875]
[None, 2.7559302, 0.46875]
[None, 2.4668293, 0.53125]
[None, 3.034531, 0.53125]
[None, 2.8796911, 0.4375]
[None, 2.3598783, 0.625]
[None, 2.5355144, 0.5]
[None, 3.8018842, 0.3125]
[None, 3.3049297, 0.375]
[None, 2.6001065, 0.5]
[None, 2.674156, 0.46875]
[None, 3.6509314, 0.40625]
[None, 2.5716412, 0.5]
[None, 2.6666732, 0.4375]
[None, 2.6111403, 0.5]
[None, 4.081786, 0.375]
[None, 2.9873228, 0.4375]
[None, 2.7122803, 0.46875]
[None, 3.5605865, 0.34375]
[None, 2.135819, 0.5625]
[None, 2.9137638, 0.40625]
[None, 3.1287436, 0.46875]
[None, 2.8168788, 0.40625]
[None, 3.157316, 0.3125]
[None, 3.0496101, 0.53125]
[None, 3.271079, 0.375]
[None, 3.3746908, 0.40625]
[None, 2.2798147, 0.59375]
[None, 2.4241247, 0.4

[None, 2.8852468, 0.46875]
[None, 3.7710862, 0.4375]
[None, 2.9339485, 0.53125]
[None, 2.5833142, 0.4375]
[None, 2.5899365, 0.53125]
[None, 2.2419333, 0.5625]
[None, 1.8501061, 0.6875]
[None, 2.204414, 0.53125]
[None, 3.0449972, 0.28125]
[None, 3.08901, 0.5]
[None, 2.7924635, 0.375]
[None, 2.2898483, 0.5625]
[None, 2.558613, 0.53125]
[None, 2.158723, 0.5]
[None, 2.5147777, 0.53125]
[None, 2.587635, 0.46875]
[None, 2.5129948, 0.5625]
[None, 1.6617398, 0.65625]
[None, 1.5815457, 0.65625]
[None, 2.7946405, 0.5625]
[None, 3.1565773, 0.375]
[None, 2.33123, 0.5625]
[None, 2.1209195, 0.53125]
[None, 3.2878342, 0.40625]
[None, 2.2063932, 0.53125]
[None, 2.2500145, 0.5]
[None, 2.7005455, 0.5]
[None, 2.1990204, 0.59375]
[None, 2.678563, 0.46875]
[None, 1.4713517, 0.8125]
[None, 3.2499447, 0.4375]
[None, 2.8089988, 0.53125]
[None, 2.3101919, 0.53125]
[None, 2.5936773, 0.625]
[None, 2.6624532, 0.53125]
[None, 1.6272023, 0.78125]
[None, 1.5357034, 0.78125]
[None, 2.3476555, 0.53125]
[None, 1.386674

KeyboardInterrupt: 

In [ ]:
batch_x.shape

In [ ]:
train_data[0:50].shape

In [ ]:
train_data[:10]

In [ ]:
y_true.dtype

In [ ]:
X.shape